# Vehicular Crashes in Longmont, Colorado

This script reads in vehicle crash data spreadsheets downloaded from CDOT. It cleans and processes the data from crashes in Longmont only and concatenates data over several years to create a single CSV time series.

Source: https://www.codot.gov/safety/traffic-safety/data-analysis/crash-data

## 1. Import Libraries

In [1]:
import pandas as pd
import numpy as np
import requests
import time
import json
import os

## 2. Preliminary Setup

In [2]:
# input file list (2021 & newer only, older files follow a different format)
files = ['CDOTRM_CD_Crash_Listing_-_2021.xlsx',
         'CDOTRM_CD_Crash_Listing_-_2022.xlsx',
         'CDOTRM_CD_Crash_Listing_-_2023.xlsx',
         'CDOTRM_CD_Crash_Listing_-_2024.xlsx'
        ]

# create an empty dataframe to populate with each year's data
all_crashes = pd.DataFrame()

# read my API key from text file for geocode.maps.co
with open('google_map_api_key.txt') as fp:
    my_api_key = fp.read().replace('\n','')

## 3. Define Functions

In [3]:
def process_crashes(crashes):
    # Save only crashes in Longmont
    crashes = crashes.loc[crashes.City == 'LONGMONT'].copy()

    # replace CUID with year-CUID
    crashes['CUID'] = crashes['Crash Date'].dt.year.astype(str)+'-'+crashes['CUID'].astype(str)

    # Fill nan values with empty string
    crashes = crashes.fillna('')
    
    return crashes

def geocode_with_google(address, api_key):
    # A function to call Google Maps API & geocode an intersection
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"
    params = {
        'address': address,
        'key': api_key
    }

    try:
        response = requests.get(base_url, params=params, timeout=10)
        response.raise_for_status()
        data = response.json()

        if data['status'] == 'OK':
            result = data['results'][0]['geometry']['location']
            return result['lat'], result['lng']
        else:
            print(f"Geocoding failed: {data['status']} - {data.get('error_message', '')}")
            return None

    except requests.RequestException as e:
        print(f"Request failed: {e}")
        return None


# Functions that return an existing coordinate or, if missing, looks up the coordinate
#    in the dictionary of geocoded intersections
def get_lat(intersection, current_lat):
    if pd.notnull(current_lat):
        return current_lat
    return geocoded_intersections.get(intersection, (None, None))[0]

def get_long(intersection, current_long):
    if pd.notnull(current_long):
        return current_long
    return geocoded_intersections.get(intersection, (None, None))[1]

## 4. Compile Data into a Single DataFrame

In [4]:
print('Processing:')

for file in files:

    print('\t'+file)
    
    # read each file into a dataframe (calamine engine is many times faster than the default
    #    openpyxl engine
    temp_df = pd.read_excel(file, header = 0, engine='calamine')

    # call function to clean data
    processed_df = process_crashes(temp_df)

    # concatenate newly processed dataframe to previously cleaned data
    all_crashes = pd.concat([all_crashes, processed_df], ignore_index = True)

print('Finished')

Processing:
	CDOTRM_CD_Crash_Listing_-_2021.xlsx
	CDOTRM_CD_Crash_Listing_-_2022.xlsx
	CDOTRM_CD_Crash_Listing_-_2023.xlsx
	CDOTRM_CD_Crash_Listing_-_2024.xlsx
Finished


## 5. Clean Data

### 5.1 Remove Spaces & Dashes from Column Names

In [5]:
all_crashes.columns = all_crashes.columns.str.replace('  ', '_') # replace double spaces with underscores
all_crashes.columns = all_crashes.columns.str.replace(' ', '_')  # replace spaces with underscores
all_crashes.columns = all_crashes.columns.str.replace('-', '')   # remove dashes
all_crashes.columns = all_crashes.columns.str.rstrip('_')        # remove trailing underscores

### 5.2 Drop Unwanted Columns

In [6]:
# Create a list of unwanted columns to drop
drop_columns = ['Agency_Id',
                'City',
                'County',
                'Rd_Section',
                'Rd_Number',
                'Record_Status',
                'Processing_Status',
                'Last_Updated',
                'Link'
               ]

all_crashes.drop(drop_columns, inplace=True, axis=1)

### 5.3 Standardize Attribute Values

#### 5.3.1 Street Names

The following code block standardizes street names by removing directions and abbreviations. For example, each of the following:

* E 9TH AVE
* E. 9TH AVE.
* 9TH AVE E
* etc

will all be standardized to 9TH AVE

In [7]:
pattern = '|'.join(['[a-zA-Z]*\. ', '\.$',                  # abbreviations
                    '\AS ',  '\AE ',  '\AW ',  '\AN ',      # Direction at start of string folled by a space
                     ' S$',   ' E$',   ' W$',   ' N$',      # Direction at end of string preceeded by a space
                     ' EB$', ' WB$', ' NB$', ' SB$'         # more street directions
                     ])

all_crashes['Location_1'] = all_crashes['Location_1'].str.replace(pattern, '', regex=True)
all_crashes['Location_2'] = all_crashes['Location_2'].str.replace(pattern, '', regex=True)

Next, street name simplifications are made. Ken Pratt goes by many names in the data set so the easiest solution is to shorten any version of that street name to just Ken Pratt. From there street types are abbreviated to consolidate versions of street names.

In [8]:
location_replacements = {
    'KEN PRATT': 'KEN PRATT',
    'BOULEVARD': 'BLVD',
    'ROAD': 'RD',
    'STREET': 'ST',
    'AVENUE': 'AVE',
    'DRIVE': 'DR',
    'CIRCLE': 'CIR',
    'COURT': 'CT'
}

for i in location_replacements:
    all_crashes.loc[all_crashes['Location_1'].str.contains(i), 'Location_1'] = location_replacements[i]
    all_crashes.loc[all_crashes['Location_2'].str.contains(i), 'Location_2'] = location_replacements[i]

all_crashes['Location_1'] = all_crashes['Location_1'].str.lstrip()
all_crashes['Location_1'] = all_crashes['Location_1'].str.rstrip()
all_crashes['Location_2'] = all_crashes['Location_2'].str.lstrip()
all_crashes['Location_2'] = all_crashes['Location_2'].str.rstrip()

#### 5.3.2 Substance Use

All variations of yes and no values for suspected alcohol, marijuana, or other drugs are replaced with simply 'Yes' or 'No'.

In [9]:
substances = ['TU1_Alcohol_Suspected', 'TU2_Alcohol_Suspected',
              'TU1_Marijuana_Suspected', 'TU2_Marijuana_Suspected',
              'TU1_Other_Drugs_Suspected', 'TU2_Other_Drugs_Suspected']

for i in substances:
    all_crashes.loc[all_crashes[i].str.contains('Yes'), i] = 'Yes'
    all_crashes.loc[all_crashes[i].str.contains('No'), i] = 'No'
    all_crashes.loc[all_crashes[i].str.contains('Marijuana Suspected'), i] = 'Yes'

## 6. Create Calculated Data Columns

### 6.1 Create New Location Columns

In order to reduce duplicate intersections, new location columns are needed such that location 1 is always less than location 2 in lexicographical order. This eliminiates duplicates such as (*3rd Ave and Main St*) and (*Main St and 3rd Ave*).

In [10]:
all_crashes['Location_A'] = all_crashes[['Location_1','Location_2']].min(axis=1)
all_crashes['Location_B'] = all_crashes[['Location_1','Location_2']].max(axis=1)
all_crashes.drop(['Location_1','Location_2'], inplace=True, axis=1)

### 6.2 Create a Column for the Roadway Intersection

This column will contain the cross strees of each accident as well as the city and state formatted such that the value may be submitted to the Google Maps API for geocoding.

In [11]:
# Add a column with the nearest intersection to the accident
all_crashes['intersection'] = all_crashes['Location_A'] + ' and ' + all_crashes['Location_B'] + ', Longmont, CO'

In [12]:
# show number of crashes w/ missing lat & long
n_missing_loc = len(all_crashes[all_crashes.iloc[:, 5]==''])
print('\nNumber of accidents with no Lat & Long: ', n_missing_loc)

# number of unique intersections to geocode
unique_intersections = all_crashes[all_crashes.iloc[:, 5]==''].intersection.unique()
n_intersections = len(unique_intersections)
print('Number of unique intersections among accidents w/out location: ', n_intersections)
print('Time to query (minutes): ', round(n_intersections*0.3/60,1))


Number of accidents with no Lat & Long:  1848
Number of unique intersections among accidents w/out location:  810
Time to query (minutes):  4.0


### 6.3 Create Column with Speeding Flag

Flag accidents where vehicle speed exceeds speed limit.

In [13]:
# Flag accidents where vehicles were speeding (1=yes, 0=no)
all_crashes['TU1_Speeding'] = np.where(all_crashes['TU1_Estimated_Speed'] > all_crashes['TU1_Speed_Limit'], 1, 0)
all_crashes['TU2_Speeding'] = np.where(all_crashes['TU2_Estimated_Speed'] > all_crashes['TU2_Speed_Limit'], 1, 0)

## 7. Use Google Maps API to Geocode Unique Intersections

In [14]:
# Name of file storing geocoded intersections
geocoded_intersections_json = 'geocoded_intersections.json'

# Check for geocoded intersection file & load it
if os.path.exists(geocoded_intersections_json):
    
    with open('geocoded_intersections.json', 'r') as f:
        geocoded_intersections = json.load(f)
    
    geocoded_intersections = {k: tuple(v) for k, v in geocoded_intersections.items()}
    
    print('Loaded geocoded intersections')
    
else:
    print('No geocoded intersection file found, geocoding w/ Google API now...')

    print('Completed:')
    
    geocoded_intersections = {}         # an empty dictionary to store intersection: (lat, long) pairs

    # variables used to print percent complete during geocoding
    quarters = [0.25, 0.5, 0.75, 1.0]
    next_quarter_index = 0
    
    for i, intersection in enumerate(unique_intersections, start=1):
        coordinates = geocode_with_google(intersection, my_api_key)
        if coordinates:
            lat = coordinates[0]
            long = coordinates[1]
            #print(coordinates) # for testing
        geocoded_intersections[intersection] = (lat, long)

        time.sleep(0.3)

        if next_quarter_index < len(quarters) and i >= n_intersections * quarters[next_quarter_index]:
            print(f"\t{int(quarters[next_quarter_index] * 100)}%")
            next_quarter_index += 1
    
    # write geocoded data to json file for later use
    with open('geocoded_intersections.json', 'w') as file:
        json.dump(geocoded_intersections, file)


No geocoded intersection file found, geocoding w/ Google API now...
Completed:
	25%
	50%
	75%
	100%


## 8. Populate Geocoded Results for Records Missing Latitude & Longitude

Now that all crash intersections have been geocoded, the latitude & longitude values in the dataframe can be populated.

In [15]:
all_crashes['Latitude'] = all_crashes.apply(
    lambda row: row['Latitude'] if pd.notnull(row['Latitude']) and row['Latitude'] != ''
    else geocoded_intersections.get(row['intersection'], (None, None))[0],
    axis=1
)

all_crashes['Longitude'] = all_crashes.apply(
    lambda row: row['Longitude'] if pd.notnull(row['Longitude']) and row['Longitude'] != ''
    else geocoded_intersections.get(row['intersection'], (None, None))[1],
    axis=1
)

## 9. Write Cleaned Data to CSV

In [16]:
# write everything to csv
all_crashes.to_csv('longmont_crashes_v02.csv', sep=',')

In [17]:
all_crashes.head()

,CUID,System_Code,City_Street,Crash_Date,Crash_Time,Latitude,Longitude,Location,Road_Description,First_HE,...,TU2_NM_Alcohol_Suspected,TU1_NM_Marijuana_Suspected,TU2_NM_Marijuana_Suspected,TU1_NM_Other_Drugs_Suspected,TU2_NM_Other_Drugs_Suspected,Location_A,Location_B,intersection,TU1_Speeding,TU2_Speeding
0,2021-40665,City Street,HOVER,2021-01-02,17:04:00,40.167007,-105.130895,On Roadway,At Intersection,Front to Side,...,,,,,,9TH AVE,HOVER ST,"9TH AVE and HOVER ST, Longmont, CO",0,0
1,2021-41331,City Street,MAINS,2021-01-03,14:19:00,40.199466,-105.104269,On Roadway,Driveway Access Related,Front to Front,...,,,,,,23RD AVE,MAIN ST,"23RD AVE and MAIN ST, Longmont, CO",0,0
2,2021-41336,City Street,3RDAV,2021-01-03,18:09:00,40.166961,-105.130811,On Roadway,At Intersection,Front to Front,...,,,,,,3RD AVE,HOVER ST,"3RD AVE and HOVER ST, Longmont, CO",0,0
3,2021-40743,City Street,KENPR,2021-01-04,11:10:00,40.141876,-105.130777,On Roadway,Non-Intersection,Front to Rear,...,,,,,,HOVER ST,KEN PRATT,"HOVER ST and KEN PRATT, Longmont, CO",0,0
4,2021-41333,City Street,WARRE,2021-01-05,09:45:00,40.174925,-105.123685,Ran off right side,Driveway Access Related,Light Pole/Utility Pole,...,,,,,,TULIP ST,WARREN AVE,"TULIP ST and WARREN AVE, Longmont, CO",0,0
